In [ ]:
#!/usr/bin/env python3
# data_preprocessing_sufiyan.py
# Extracted preprocessing workflow for Sufiyan Khan

from pathlib import Path
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')

# ---- CONFIG ----
INPUT_XLSX = 'Final_Passenger_Data.xlsx'   # change path if needed
SHEET_TO_USE = 0                           # index or sheet name
# ----------------

# 1) Load file and inspect
xlsx = pd.ExcelFile(INPUT_XLSX)
print("Sheets found:", xlsx.sheet_names)

df = pd.read_excel(INPUT_XLSX, sheet_name=SHEET_TO_USE)
print("Columns:", df.columns.tolist())
print("Rows:", len(df))
print(df.head(5))

# 2) Detect and parse date column
date_candidates = [c for c in df.columns 
                   if 'date' in c.lower() or 'day' in c.lower() or 'time' in c.lower()]
parsed_date_col = None

for c in date_candidates:
    try:
        tmp = pd.to_datetime(df[c], errors='coerce')
        if tmp.notna().sum() > 0:
            parsed_date_col = c
            df[c] = tmp
            break
    except Exception:
        pass

if parsed_date_col is None:
    for c in df.columns:
        try:
            tmp = pd.to_datetime(df[c], errors='coerce')
            if tmp.notna().sum() > len(df) * 0.5 and tmp.notna().sum() > 10:
                parsed_date_col = c
                df[c] = tmp
                break
        except Exception:
            pass

if parsed_date_col is None:
    raise SystemExit("Couldn't detect a date column automatically.")

print("Using date column:", parsed_date_col)

df = df.rename(columns={parsed_date_col: 'date'})
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df = df.dropna(subset=['date'])

# 3) Detect passenger/count column
candidates = [c for c in df.columns 
              if 'pass' in c.lower() or 'pax' in c.lower() 
              or 'count' in c.lower() or 'num' in c.lower()]

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for c in numeric_cols:
    if c not in candidates and df[c].min() >= 0 and df[c].mean() > 0 and df[c].nunique() > 3:
        candidates.append(c)

candidates = list(dict.fromkeys(candidates))
print("Passenger candidate columns:", candidates)

sums = {
    c: df[c].fillna(0).sum() if np.issubdtype(df[c].dtype, np.number) else -1
    for c in candidates
}
passenger_col = max(sums, key=sums.get)
print("Chosen passenger column:", passenger_col)

df[passenger_col] = pd.to_numeric(df[passenger_col], errors='coerce').fillna(0)

# 4) Create monthly aggregated dataset
df['month_start'] = df['date'].dt.to_period('M').dt.to_timestamp()

monthly = (
    df.groupby('month_start')[passenger_col].sum()
      .reset_index()
      .rename(columns={'month_start': 'ds', passenger_col: 'y'})
)

monthly = monthly.sort_values('ds').set_index('ds')

# Fill missing months
all_months = pd.date_range(monthly.index.min(), monthly.index.max(), freq='MS')
monthly = monthly.reindex(all_months).fillna(0)
monthly.index.name = 'ds'

print("Monthly series from", monthly.index.min().date(), 
      "to", monthly.index.max().date(), "(", len(monthly), "months )")

# Optional: save the cleaned monthly dataset
monthly.to_csv("clean_monthly_passenger_series.csv")
print("Saved cleaned monthly dataset to: clean_monthly_passenger_series.csv")

